In [2]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import re
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import csv

# 1. Đọc dữ liệu từ file CSV
data = pd.read_csv('./dataset/Amazon_Product_Review_cleaned.csv')

# Hiển thị các cột của dataset (để kiểm tra)
print("Các cột trong dataset:", data.columns.tolist())

# 2. Tiền xử lý văn bản trong cột 'review'
def preprocess_text(text):
    text = str(text)            # Đảm bảo text là chuỗi
    text = text.lower()         # Chuyển về chữ thường
    # Giữ lại các ký tự chữ cái (bao gồm có dấu), số và dấu cách
    text = re.sub(r'[^a-zA-Z0-9À-ỹ\s]', '', text)
    return text

# Áp dụng tiền xử lý cho cột review
data['review'] = data['review_headline'].apply(preprocess_text)

# 3. Loại bỏ các dòng có giá trị bị thiếu
data = data.dropna(subset=['review', 'star_rating'])

# 4. Đảm bảo cột star_rating ở dạng số nguyên
data['star_rating'] = data['star_rating'].astype(int)

# Hiển thị các giá trị star_rating độc nhất
print("Các star_rating độc nhất:", data['star_rating'].unique())
# 1. Tách dữ liệu train-test
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['star_rating'], test_size=0.2, random_state=42)

# 2. Vector hóa văn bản bằng TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 3. Huấn luyện mô hình Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# 4. Dự đoán và đánh giá
y_pred_nb = nb_model.predict(X_test_tfidf)

print("🔹 Accuracy (Naive Bayes):", accuracy_score(y_test, y_pred_nb))
print("🔹 Classification Report:\n", classification_report(y_test, y_pred_nb))
print("🔹 Confusion Matrix:\n", confusion_matrix(y_test, y_pred_nb))


Các cột trong dataset: ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date', 'sentiment']
Các star_rating độc nhất: [5 1 3 4 2]
🔹 Accuracy (Naive Bayes): 0.7568881685575365
🔹 Classification Report:
               precision    recall  f1-score   support

           1       0.73      0.32      0.44       344
           2       0.93      0.11      0.20       237
           3       0.73      0.29      0.41       462
           4       0.68      0.44      0.54      1159
           5       0.77      0.98      0.86      3968

    accuracy                           0.76      6170
   macro avg       0.77      0.43      0.49      6170
weighted avg       0.75      0.76      0.72      6170

🔹 Confusion Matrix:
 [[ 109    0   15   46  174]
 [  21   26   14   41  135]
 [   9    1  132   91  229]
 [   4    0   17  514  